In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect, desc, asc
from sqlalchemy.sql import label
from flask import Flask, jsonify
import matplotlib.pyplot as plt 
import seaborn as sns

In [ ]:
# Database setup
engine = create_engine("sqlite:///hawaii.sqlite")

In [ ]:
# Declare auto base (Reflection)
Base = automap_base()

In [ ]:
# Reflect the database tables
Base.prepare(engine, reflect=True)

In [ ]:
Base.classes.keys()
inspector = inspect(engine)

# Collect the names of tables within the database
inspector.get_table_names()

In [ ]:
# Display 15 rows from the measurements table
engine.execute('SELECT * FROM Measurements LIMIT 10').fetchall()

In [ ]:
Station = Base.classes.Stations
Measurement = Base.classes.Measurements

In [ ]:
columns = inspector.get_columns('measurements')
for c in columns:
    print(c['name'], c["type"])

In [ ]:
session = Session(engine)

In [ ]:
engine.execute('SELECT * FROM stations LIMIT 15').fetchall()

# Precipitation Analysis

In [ ]:
last_date = session.query(Measurement.date,Measurement.prcp).order_by(Measurement.date.desc()).first()
print(last_date)

In [ ]:
last_year = dt.date(2017, 8, 23) - dt.timedelta(days=365)
print(last_year)

In [ ]:
result = session.query(Measurement.date, Measurement.prcp).\
    filter(Measurement.date > last_year).\
    order_by(Measurement.date).all()

In [ ]:
df = pd.DataFrame(result, columns=['date', 'precipitation'])
df.set_index(df['date'], inplace=True)
df.tail()

In [ ]:
df.plot(x_compat=True, color='#003399') 
plt.xticks(rotation='45')
plt.ylim(0,8,2)
plt.show()
plt.savefig('rain_station.png')

# Station Analysis

In [ ]:
session.query(Measurement.station, func.sum(Measurement.station))\
.group_by(Measurement.station).all()

In [ ]:
# Count the number of stations in the Measurement table
locations = session.query(Measurement).group_by(Measurement.station).count()
print("There are {} stations.".format(locations))

In [ ]:
# find the station with the most temperature observations
busy_station = session.query(Measurement.station, func.count(Measurement.tobs)).group_by(Measurement.station).\
               order_by(func.count(Measurement.tobs).desc()).all()

busiest = busy_station[0][0]    
print("The busiest Station:",busiest,busy_station[0][1])
for station, count in busy_station:
    print("Station:",station,count)

In [ ]:
# query to pull the last year of precipitation data for the busiest station
temperature = session.query(Measurement.station, Measurement.date, Measurement.tobs).\
    filter(Measurement.station == busiest).\
    filter(Measurement.date > last_year).\
    order_by(Measurement.date).all()

In [ ]:
# plot the temperature data in a histogram with 12 bins
temp_df=pd.DataFrame(temperature)
plt.hist(temp_df['tobs'],12,color='#003399')
plt.xlabel("Temperature")
plt.ylabel("Frequency")
plt.title("Station Analysis")
plt.savefig('station_analysis.png')
plt.show()

# Temperature Analysis

In [ ]:
def calc_temps(startdate, enddate):
    minimum = session.query(func.min(Measurement.tobs)).filter(Measurement.date > startdate)\
                             .filter(Measurement.date < enddate).all()
    maximum = session.query(func.max(Measurement.tobs)).filter(Measurement.date > startdate)\
                             .filter(Measurement.date < enddate).all()
    average = session.query(func.avg(Measurement.tobs)).filter(Measurement.date > startdate)\
                             .filter(Measurement.date < enddate).all()
    return minimum, maximum, average

In [ ]:
trip = calc_temps("2017-01-01","2018-01-01")
trip

In [ ]:
df = pd.DataFrame({'tmin': trip[0][0], 'tmax': trip[1][0], 'avg': trip[2][0]}, columns=['tmin','avg','tmax'])
df

In [ ]:
error = df['tmax'].sub(df['tmin'])
df.plot.bar(y='avg',yerr=error, color='#00babc')
plt.title("Trip Average Temp")
plt.savefig('average_trip_temperature.png')
plt.show() 

# Climate App

In [ ]:
date = dt.datetime(2018, 5, 31)

In [ ]:
today = dt.date.today

In [ ]:
app = Flask(__name__)

In [ ]:
@app.route("/")
def welcome():
    """List all available api routes."""
    return (
        f"Available Routes:<br/>"
        f"/api/v1.0/precipitation<br/>"
        f"/api/v1.0/stations<br/>"
        f"/api/v1.0/tobs<br/>"
        f"/api/v1.0/stations</br>"
        f"/api/v1.0/<start></br>"
        f"/api/v1.0/<start>/<end>"
    )

In [ ]:
@app.route("/api/v1.0/precipitation")
def precipitation():
    dict = {}
    data = df[(df.date >= str(last_year)) & (df.date <= str(today))].groupby(['date']).sum()
    for index, row in df.iterrows():
        dict[row.date] = row.prcp
    return jsonify(dict)

In [ ]:
@app.route("/api/v1.0/stations")
def stations():
    stations = list(df.measurement.unique())
    return jsonify(stations)

In [ ]:
@app.route("/api/v1.0/tobs")
def tobs():
    results = session.query(measurement.date, measurement.tobs).filter(measurement.date >= last_year).all()
    temp_results = list(np.ravel(results))

    return jsonify(temp_results)

In [ ]:
@app.route("/api/v1.0/<start>")
def temp_start(start):
    temps = df[(df.date >= start)]
    min_temp = temps.tobs.min()
    max_temp = temps.tobs.max()
    mean_temp = temps.tobs.mean()
    return jsonify({"min_temp": int(min_temp), "max_temp": int(max_temp), "mean_temp": int(mean_temp)})

In [ ]:
if __name__ == '__main__':
    app.run(debug=False)